In [3]:
import pandas as pd
import numpy as np
import sqlite3
import math

from numpy import transpose
from numpy.linalg import inv,pinv, det
from scipy.stats import norm

engine = sqlite3.connect('yahoo')
np.set_printoptions(precision=3)
click_cols = ['time', 'user_1', 'user_2', 'user_3', 'user_4', 'user_5', 'user_6', 'displayed', 'clicked']

In [4]:
articles_df = pd.read_sql_query('SELECT * FROM articles',con=engine).set_index('index')
clicks_df = pd.read_sql_query('''
    SELECT * FROM clicks WHERE (ABS(CAST((RANDOM()) as int)) % 100) < 10 AND length(article_pool) < 210
''', con=engine)
clicks_df[click_cols] = clicks_df[click_cols].apply(pd.to_numeric, ) # Temporary workaround, this should be done before sql

In [ ]:
#articles_df.head()

In [5]:
clicks_df.shape

(150494, 11)

In [ ]:
# clicks_df.head()

In [6]:
def eta(T):
    """ 
    Generates the cutoff probabilities for exploration rounds in interval chaining. 
    
    :param T: the total number of iterations
    """
    return np.array([pow(t, -1/3) for t in range(1,T+1)])

In [36]:
clicks_df.as_matrix(['displayed', 'clicked'])[0,1]

0

In [34]:
np.append(np.empty((0,1)), 1)

array([ 1.])

In [38]:
user_feat_mat = clicks_df.as_matrix(['user_1', 'user_2', 'user_3', 'user_4', 'user_5', 'user_6'])
clicked_mat = clicks_df.as_matrix(['clicked'])

def select_logged_event(t, k, d, X, Y):
    # Extract all article vectors for first log item.
    _article_pool = eval(clicks_df.ix[t]['article_pool'])
    _article_vecs = np.array([articles_df.ix[x].values for x in _article_pool])
    # For each of the article row vectors, add a copy of the user features.
    logged = np.append(_article_vecs, values=np.tile(user_feat_mat[t], (len(_article_vecs), 1)), axis=1)
#     rewards = np.array(list(map(lambda x : 1 if int(x) == disp_mat[t] else 0, _article_pool)))
    return np.vstack([X, logged.reshape((1, k, d))*1000]), np.append(Y, clicked_mat[t])

In [8]:
def ols_intervals(X, Y, k, _delta, t, T):
    intervals = []
    for i in range(k):
        # Compute beta hat.
        _Xti = X[:t+1,i]
        _XtiT = transpose(_Xti)
        try:
            _XTX = pinv(_XtiT.dot(_Xti))
        except:
            print('Encountered singular matrix. Ignoring.')
            continue
        _Yti = Y[:t+1,i]
        Bh_t_i = _XTX.dot(_XtiT).dot(_Yti)  # Compute OLS estimators.
        yh_t_i = Bh_t_i.dot(X[t,i])
        _s2 = np.var(Y[:t+1,i])
        # Compute the confidence interval width using the inverse CDF.
        w_t_i = norm.ppf(1 - _delta/(2*T*k), loc=0, 
                         scale=np.sqrt(_s2 * X[t,i].dot(_XTX).dot(transpose(X[t,i]))))
        if not math.isnan(w_t_i):
            intervals.append([yh_t_i - w_t_i, yh_t_i + w_t_i])
    return intervals

In [ ]:
"""
Potentially big issue.
Since we can only really learn on the samples that we've observed results for...
we need to independently keep track of the samples observed for EACH arm. 
The sum of all |X_{t,i}| over all i then must equal t.
"""

In [19]:
def top_interval_yahoo(T):
    """
    Simulates T rounds of TopInterval for k using the evaluation framework
    described in https://arxiv.org/pdf/1003.5956.pdf.
    
    Evaluation Protocol
    ===================
    Let σ be an input stream of logged events to be used for the simulation,
    where an event consists of a tuple consisting of the user vector, 
    20 article vectors (context vectors), the displayed vector (selected arm), 
    and the result (observed reward).
    Step through each element s sampled from the stream.
    Let t denote the current time-step in the algorithm and h_{t-1} prior history.
    If given h_{t-1} and the current context, the algorithm picks the same vector
    as the selected arm - retain the event and add it to the history. Otherwise,
    keep processing items.
    
    :param T: the number of iterations
    """
    k = 20
    d = 12
    X = np.empty((0, k, 12))
    Y = np.empty((0,20))
    _delta = 0.1
    
    _eta = eta(T)                               # exploration cutoff probabilities
    cursor = 0
    performance = []
    for t in range(T):
    
        if t <= d or np.random.rand() <= _eta[t]:
            # Play uniformly at random from [1, k].
#             print('Exploration round.')
            X, Y = select_logged_event(cursor, k, d, X, Y)
            cursor += 1
            performance.append(1)
            print('Iteration [{0} / {1}]'.format(t, T))
        else:
            while (cursor <= len(clicks_df)):
                # Compute input vector for next item in stream and corresponding reward vector.
                Xp, Yp = select_logged_event(cursor, k, d, X, Y)
                intervals = ols_intervals(Xp, Yp, k, _delta, t, T)
                # Pick the agent with the largest upper bound.
                pick = np.argmax(np.array(intervals)[:,1])
#                 print('Intervals: {0}'.format(intervals))
                _article_pool = eval(clicks_df.ix[cursor]['article_pool'])
                print('Pick: {0}. Displayed: {1}'.format(_article_pool[pick], clicks_df.ix[t]['displayed']))
                if clicks_df.ix[cursor]['displayed'] == int(_article_pool[pick]):
                    X, Y = Xp, Yp
#                     print('Iteration [{0} / {1}], matched with stream! {2}'.format(t, T, Yp[t, pick]))
                    performance.append(Yp[t, pick])
                    cursor += 1
                    break
                cursor += 1
            
    # Compute sum of best picks over each iteration.
    best = [Y[i].max() for i in range(2, T)]
    print('Best: {0}'.format(sum(best)))
    print('Performance: {0}'.format(sum(performance)))
    print('Regret: {0}'.format(sum(best) - sum(performance)))
    print(X)


In [23]:
tmp = top_interval_yahoo(100)

Iteration [0 / 100]
Iteration [1 / 100]
Iteration [2 / 100]
Iteration [3 / 100]
Iteration [4 / 100]
Iteration [5 / 100]
Iteration [6 / 100]
Iteration [7 / 100]
Iteration [8 / 100]
Iteration [9 / 100]
Iteration [10 / 100]
Iteration [11 / 100]
Iteration [12 / 100]
Iteration [13 / 100]
Pick: 109473. Displayed: 109494
Pick: 109509. Displayed: 109494
Pick: 109473. Displayed: 109494
Pick: 109503. Displayed: 109494
Iteration [15 / 100]
Iteration [16 / 100]
Pick: 109503. Displayed: 109503
Pick: 109484. Displayed: 109503
Pick: 109506. Displayed: 109503
Pick: 109502. Displayed: 109503
Pick: 109484. Displayed: 109503
Pick: 109503. Displayed: 109503
Pick: 109502. Displayed: 109503
Pick: 109510. Displayed: 109503
Pick: 109510. Displayed: 109503
Pick: 109510. Displayed: 109503
Pick: 109510. Displayed: 109503
Pick: 109510. Displayed: 109503


/Users/jtcho/Jupyter/cis625/fairml/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1907: RuntimeWarning: invalid value encountered in multiply
  lower_bound = self.a * scale + loc
/Users/jtcho/Jupyter/cis625/fairml/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1908: RuntimeWarning: invalid value encountered in multiply
  upper_bound = self.b * scale + loc


Pick: 109495. Displayed: 109503
Pick: 109514. Displayed: 109503
Pick: 109503. Displayed: 109503
Pick: 109484. Displayed: 109503
Pick: 109503. Displayed: 109503
Pick: 109506. Displayed: 109503
Pick: 109502. Displayed: 109503
Pick: 109510. Displayed: 109503
Pick: 109509. Displayed: 109503
Pick: 109506. Displayed: 109503
Pick: 109506. Displayed: 109503
Pick: 109509. Displayed: 109503
Pick: 109495. Displayed: 109503
Pick: 109503. Displayed: 109503
Pick: 109494. Displayed: 109503
Pick: 109492. Displayed: 109503
Pick: 109502. Displayed: 109503
Pick: 109510. Displayed: 109503
Pick: 109494. Displayed: 109503
Pick: 109509. Displayed: 109503
Pick: 109503. Displayed: 109503
Pick: 109503. Displayed: 109503
Pick: 109503. Displayed: 109503
Pick: 109510. Displayed: 109503
Pick: 109506. Displayed: 109503
Pick: 109492. Displayed: 109503
Pick: 109494. Displayed: 109503
Pick: 109502. Displayed: 109503
Pick: 109494. Displayed: 109503
Pick: 109508. Displayed: 109503
Pick: 109503. Displayed: 109503
Pick: 10

Pick: 109505. Displayed: 109508
Pick: 109506. Displayed: 109508
Pick: 109494. Displayed: 109508
Pick: 109502. Displayed: 109508
Pick: 109506. Displayed: 109508
Pick: 109506. Displayed: 109508
Pick: 109509. Displayed: 109508
Pick: 109484. Displayed: 109508
Pick: 109506. Displayed: 109508
Pick: 109495. Displayed: 109508
Pick: 109494. Displayed: 109508
Pick: 109510. Displayed: 109508
Pick: 109501. Displayed: 109508
Pick: 109505. Displayed: 109508
Pick: 109492. Displayed: 109508
Pick: 109484. Displayed: 109508
Pick: 109506. Displayed: 109508
Pick: 109506. Displayed: 109508
Pick: 109506. Displayed: 109508
Pick: 109494. Displayed: 109508
Pick: 109514. Displayed: 109508
Pick: 109510. Displayed: 109508
Pick: 109498. Displayed: 109508
Pick: 109505. Displayed: 109512
Pick: 109509. Displayed: 109512
Iteration [22 / 100]
Pick: 109505. Displayed: 109498
Pick: 109473. Displayed: 109498
Iteration [24 / 100]
Pick: 109492. Displayed: 109501
Pick: 109515. Displayed: 109501
Pick: 109484. Displayed: 10950

Pick: 109473. Displayed: 109511
Pick: 109473. Displayed: 109511
Pick: 109473. Displayed: 109511
Pick: 109473. Displayed: 109511
Pick: 109473. Displayed: 109511
Pick: 109473. Displayed: 109511
Pick: 109473. Displayed: 109511
Pick: 109473. Displayed: 109511
Pick: 109473. Displayed: 109511
Pick: 109473. Displayed: 109511
Pick: 109473. Displayed: 109511
Pick: 109473. Displayed: 109515
Pick: 109505. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109501. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109501. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109473
Pick: 10

Pick: 109473. Displayed: 109512
Pick: 109473. Displayed: 109512
Pick: 109473. Displayed: 109512
Pick: 109473. Displayed: 109512
Pick: 109473. Displayed: 109512
Pick: 109473. Displayed: 109512
Pick: 109473. Displayed: 109512
Pick: 109473. Displayed: 109512
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 109473. Displayed: 109505
Pick: 10

Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109498. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 109473. Displayed: 109515
Pick: 10

Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 109473. Displayed: 109502
Pick: 10

## Examining Malformed Data

I observed that some of the event log items have > 20 articles in the article pool, which creates a problem since we assume a constant $k$ arms across all events.

It turns out that there are around 2.53 million such events out of the 4.68 million in the first batch file, which is a lot! 

Additionally, it is worth noting that there are a good number of articles that have identical feature vectors despite having different labels.

However, it is not as simple as removing the duplicates for all logged event items, as I determined at least one such case where doing so would result in < 20 articles in the pool. Therefore, distinct articles are distinct despite having identical article vectors (which is troublesome). 

A workaround I am using now is simply to only select the events with 20 articles, which seems to work in the interim.

In [ ]:
pd.read_sql_query('''SELECT * FROM clicks WHERE time=1241162400 AND displayed=109513 AND user_1=0.001766''', con=engine)

In [ ]:
pd.read_sql_query('''SELECT COUNT(*) FROM clicks''', con=engine)

In [ ]:
pd.read_sql_query('''SELECT COUNT(*) FROM clicks''', con=engine)
bad_df = pd.read_sql_query('''SELECT * FROM clicks WHERE length(article_pool) >= 210''', con=engine)
len(bad_df)

In [ ]:
bad_sample = eval(bad_df.ix[80840]['article_pool'])

In [ ]:
dupes = pd.DataFrame(articles_df.duplicated(), columns=['Result'])
articles_df.ix[list(dupes[dupes.Result == True].index)]

In [12]:
# I've constructed an inverse index to allow swift de-duplication of articles with identical vectors.
d1 = {key:'109509' for key in ['109509', '109503', '109494', '109525', '109530', '109533', '109545']}
d2 = {key:'109506' for key in ['109506', '109547', '109550']}
d3 = {key:'109527' for key in ['109527', '109531', '109543']}
dupe_index = {**d1, **d2, **d3}

In [ ]:
print(len(bad_sample))
print(len(set(map(lambda x : dupe_index[x] if x in dupe_index else x, bad_sample))))

In [14]:
dupe_index['109509']

'109509'